In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from datetime import datetime
import time
import random
from django.db.models import Q

# Full Data Crawler

In [ ]:
board_errors = []
post_errors = []

try:
    for offset_start in tqdm(range(0, 8419)):
        urls = 'https://okky.kr/articles/tech-qna?offset={}&max=24&sort=id&order=desc'.format(offset_start*24)
        print(urls)

        post_urls = get_post_urls(urls)
        try:
            for post_url in post_urls:
                post_info = get_post_info(post_url)

                create(post_url, post_info)

            time.sleep( random.uniform(1,15) )
        except AttributeError as error:
            print('Attribute Error')
            print(post_url)
            pass
        except:
            print('Post Error')
            print(post_url)
            post_errors.append(post_url)
            time.sleep(30)
except:
    print('Board Error')
    print(urls)
    board_errors.append(urls)
    time.sleep(60)

print('Board Errors: 'board_error)
print('Post Errors: 'post_error)

### 게시판 게시글 URL 크롤링

In [ ]:
def get_post_urls(urls):
    f = requests.get(urls)
    soup = bs(f.text, "html.parser")

    ul_tags = soup.find_all('ul',class_='list-group')[1]
    
    none_comments_li_tags = ul_tags.find_all('li', class_='list-group-no-note')
    unchecked_comments_li_tags = ul_tags.find_all('li', class_='list-group-has-note')
    checked_comments_li_tags = ul_tags.find_all('li', class_='list-group-success')
    
    full_urls = []
    
    #댓글이 없는 게시글 = list-group-no-note clearfix
    for none_comments_li_tag in none_comments_li_tags:
        a_tags = none_comments_li_tag.find_all("a")
        for a_tag in a_tags:
            urls = a_tag['href']
            if '/article/' in urls:
                full_url = "https://okky.kr" + urls
                
                full_urls.append(full_url)
    
    #답변이 있는 게시글 = list-group-has-note
    for unchecked_comments_li_tag in unchecked_comments_li_tags:
        a_tags = unchecked_comments_li_tag.find_all("a")
        for a_tag in a_tags:
            urls = a_tag['href']
            if '/article/' in urls:
                full_url = "https://okky.kr" + urls
                
                full_urls.append(full_url)
    
    #질문자가 채택한 답변이 있는 게시글 = list-group-success
    for checked_comments_li_tag in checked_comments_li_tags:
        a_tags = checked_comments_li_tag.find_all("a")
        for a_tag in a_tags:
            urls = a_tag['href']
            if '/article/' in urls:
                full_url = "https://okky.kr" + urls
                
                full_urls.append(full_url)
    
    return full_urls

### 게시글 크롤링

In [ ]:
def get_post_info(post_url):
    f = requests.get(post_url)
    soup = bs(f.text, "html.parser")
    
    #게시글 제목
    post_title = soup.find('h2', class_='panel-title').text.strip()
    
    #게시글 댓글 수
    comment_count = soup.find('h3', class_='panel-title').text.strip()
    total_comment = int(comment_count[3:])
    
    #게시글 게시 날짜
    post_date_time_obj = soup.find('span', class_="timeago").text.strip()
    post_datetime = datetime.strptime(post_date_time_obj, '%Y-%m-%d %H:%M:%S')
    
    #게시글 내용
    post_content = str(soup.find("article", class_="content-text"))
    
    #게시글 구성
    articles = soup.find_all('article')

    post_component = ''

    for article in articles:
        post_component += str(article)

    soup1 = bs(post_component, 'html.parser')

    img = soup1.find('img')
    code =  soup1.find('pre')

    if img and code:
        post_component = 'img$code'
    elif img:
        post_component = 'img'
    elif code:
        post_component = 'code'
    else:
        post_component = 'text'
    
    #게시글 코드 언어
    codes = soup.find_all('code')

    code_lang = ''

    if len(codes) == 0:
        code_lang = ''
    else:
        for i in codes:
            if soup.find('code',class_='파이썬'):
                continue
            elif soup.find('code',class_='gatsby-code-jsx'):
                startindex = str(i).find('class="')+len('class="gatsby-code-')
                endindex = str(i)[startindex:].find('"')
            elif soup.find('code',class_='hljs'):
                continue
            elif soup.find('code',class_='html'):
                continue
            elif soup.find('code',class_='jsx-d60897d45df4a3d0'):
                continue
            elif soup.find('code',class_=''):
                continue
            else:
                startindex = str(i).find('class="')+len('class="language-')
                endindex = str(i)[startindex:].find('"')

            lang = str(i)[startindex:endindex+startindex]

            if lang == 'null':
                pass
            elif lang not in code_lang:
                code_lang = code_lang + '$' + lang
    
    return_value = [post_title, total_comment, post_datetime, post_content, post_component, code_lang]
    
    #게시글 채택 답변
    if soup.find('i',class_='fa fa-check'):
        
        answers = soup.find_all("li", class_='list-group-item note-item clearfix')
        for answer in answers:
            if answer.find('i',class_='fa fa-check'):
                checked_answer = str(answer.find('article'))
                
                checked_answer_datetime = answer.find('span',class_='timeago').text
                answer_datetime = datetime.strptime(checked_answer_datetime, '%Y-%m-%d %H:%M:%S')
                
                return_value.append(checked_answer)
                return_value.append(answer_datetime)
            else:
                pass
    else:
        checked_answer = ''
        answer_datetime = datetime.strptime('0001-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        
        return_value.append(checked_answer)
        return_value.append(answer_datetime)
    
    return return_value

In [ ]:
def create(post_url, post_info):
    QuestionTable.objects.create(
        post_url = post_url,
        post_title = post_info[0],
        total_comment = post_info[1],
        post_datetime = post_info[2],
        post_content = post_info[3],
        post_component = post_info[4],
        code_lang = post_info[5],
        checked_answer = post_info[6],
        answer_datetime = post_info[7]
    )
    print('New Post:', post_url)